# let's look for a highly-expressed NNC that also has protein change

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [3]:
def clean_figure(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis="x", rotation=45)

In [4]:
meta = load_meta()
meta = meta.loc[meta.merged_run_mode==True]
sample_d = dict([(entry.cell_line_id, entry['sample']) \
                 for ind, entry in meta.iterrows()])
    
    

f = expand(proc_cfg(config['lr']['kallisto']['quant']['merge_matrix_tpm_tsv'],od))[0]
df = pd.read_csv(f, sep='\t')
df.head()
df.columns = [d if d == 'transcript_id' else d.split('_')[0] for d in df.columns]
df.rename({'transcript_id':'tid'}, axis=1, inplace=True)
df.rename(sample_d, axis=1, inplace=True)

In [5]:
# get max expression value per transcript
df = df.set_index('tid')
df['max_cpm'] = df.max(axis=1)

In [6]:
mt_df = pd.read_csv('../data/05_mastertable/poder_master_table_fixed_genics.tsv', sep='\t')
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)

In [7]:
# protein stuff
p_df = pd.read_csv('241120_long_struct_cat_aa_cat.tsv', sep='\t')

# only novel 
p_df = p_df.loc[p_df.aa_seq_novelty=='Novel']

In [8]:
# min expression 
min_cpm = 5
df = df.loc[df.max_cpm>=min_cpm]

In [9]:
# nncs
df = df.reset_index()
print(len(df.index))
df = df.loc[df.tid.isin(mt_df.loc[mt_df.structural_category=='NNC', 'isoform'].tolist())]
print(len(df.index))

14647
1885


In [10]:
# novel things w/ novel proteins
print(len(df.index))
df = df.loc[df.tid.isin(p_df.isoform.tolist())]
print(len(df.index))

1885
844


In [11]:
# add gene IDs so I can look at these genes
df = df.merge(mt_df[['isoform', 'geneid.v']],
                how='left',
                left_on='tid',
                right_on='isoform')

In [15]:
df.sort_values(by='max_cpm', ascending=False).head(10)

,tid,YRI5,YRI6,YRI7,HAC1,HAC2,HAC3,HAC4,HAC5,HAC6,...,CEU3,CEU4,CEU5,MPC4,YRI1,YRI2,YRI3,max_cpm,isoform,geneid.v
100,transcript_102081,56.43510,87.271900,80.36070,97.4697,103.17500,89.34290,64.40390,64.11830,102.88300,...,103.1680,96.46960,1.221200e+02,413.258000,98.13340,107.775000,40.779600,509.266,transcript_102081,ENSG00000197756.10
525,transcript_229410,194.05700,15.505600,88.59410,90.6736,191.88300,161.90400,60.55930,162.96800,86.53800,...,115.2880,130.31200,1.476450e+02,16.051800,100.35500,116.536000,115.216000,299.215,transcript_229410,ENSG00000150991.16
304,transcript_346879,1.89236,0.167479,1.81594,81.2511,0.00561,194.83400,1.76788,1.29289,127.61700,...,23.8188,1.95665,2.325380e-09,0.498969,1.29124,0.037405,0.857945,292.013,transcript_346879,ENSG00000196126.12
737,transcript_282857,188.65100,188.076000,181.17000,160.7310,187.54600,130.20600,140.29200,186.38700,146.43800,...,214.3660,157.56500,2.002790e+02,237.394000,101.75500,201.705000,101.022000,265.157,transcript_282857,ENSG00000105193.9
542,transcript_239500,62.09700,62.631800,234.88000,191.6920,38.41340,61.58260,69.39240,63.17220,63.60490,...,81.2370,86.85120,9.901150e+01,88.808900,215.23300,58.220400,91.337900,234.880,transcript_239500,ENSG00000133112.17
101,transcript_102083,27.23180,38.400600,35.25290,41.4823,42.39240,42.46830,30.16310,32.16950,44.75230,...,54.5053,43.31130,5.509550e+01,192.145000,44.48110,47.102700,19.333500,231.366,transcript_102083,ENSG00000197756.10
138,transcript_134953,117.34000,158.468000,122.40500,114.9070,117.04600,123.17200,138.98000,127.63000,168.14100,...,155.9970,123.23600,1.259320e+02,221.789000,127.11500,161.305000,142.404000,221.789,transcript_134953,ENSG00000232112.3
535,transcript_239361,37.50690,38.489100,174.97300,116.6900,28.58120,44.89380,53.49450,33.65030,44.30480,...,41.8343,61.28210,6.530810e+01,63.753100,104.79900,35.484800,51.678300,174.973,transcript_239361,ENSG00000133112.17
46,transcript_207293,2.96905,6.906950,153.15000,170.1770,3.45322,5.01338,31.36270,3.66373,7.30984,...,55.1088,52.07720,7.292640e+01,146.927000,112.84600,2.122320,35.716300,173.801,transcript_207293,ENSG00000116251.11
461,transcript_33803,110.92300,92.679600,85.51630,146.7660,31.92650,115.85900,96.53160,149.99000,89.30160,...,154.6740,120.06800,1.248300e+02,41.370500,118.82100,116.032000,76.395800,166.055,transcript_33803,ENSG00000177600.10


In [13]:
df

,tid,YRI5,YRI6,YRI7,HAC1,HAC2,HAC3,HAC4,HAC5,HAC6,...,CEU3,CEU4,CEU5,MPC4,YRI1,YRI2,YRI3,max_cpm,isoform,geneid.v
0,transcript_11288,0.824280,0.722806,0.000000,0.000000,0.946322,0.514791,0.849513,0.905785,0.839436,...,1.407110,2.633860,1.448400,1.940460,1.205510,0.674467,1.316290,5.13914,transcript_11288,ENSG00000054282.16
1,transcript_1667,0.000000,0.279376,0.107144,0.144927,0.222129,0.165249,0.401112,0.094033,0.000000,...,0.000000,0.122660,0.084622,0.000000,0.328226,0.076182,0.394974,18.32370,transcript_1667,ENSG00000064886.14
2,transcript_6058,2.379530,2.018300,2.058910,1.139050,3.823520,4.694190,3.405580,3.851690,4.166660,...,3.255720,2.789130,3.823130,0.739017,2.710830,2.695930,3.487930,6.42632,transcript_6058,ENSG00000072694.22
3,transcript_12208,1.244680,3.632200,8.091200,2.507410,1.327680,5.088190,5.646510,2.260810,3.796290,...,3.526090,3.435090,3.154800,4.946250,2.617120,2.788730,11.057600,11.05760,transcript_12208,ENSG00000090273.14
4,transcript_12212,0.638518,2.283400,2.161360,1.893690,1.179290,1.512120,1.887090,1.301280,1.191090,...,0.519564,2.866770,1.578080,3.078970,1.210020,1.345540,4.326680,5.45017,transcript_12212,ENSG00000090273.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,transcript_380569,1.317280,2.470770,0.439373,0.842221,2.866520,1.581960,2.869110,1.830730,2.046370,...,3.168480,2.477630,2.515660,6.885170,1.951650,1.317370,2.256330,6.88517,transcript_380569,ENSG00000198157.11
840,transcript_189933,1.727640,0.000000,0.000000,0.000000,4.123090,0.000000,4.152280,0.000000,3.106910,...,3.236730,1.774490,0.000000,3.443970,2.322330,0.000000,0.000000,5.76161,transcript_189933,ENSG00000198692.10
841,transcript_189954,1.812510,0.000000,0.000000,0.000000,2.655900,0.000000,2.016160,0.000000,3.242370,...,3.233580,1.849100,0.000000,2.508110,2.186710,0.000000,0.000000,6.53964,transcript_189954,ENSG00000198692.10
842,transcript_190157,1.715910,30.997400,0.149749,10.424600,23.465500,7.545030,19.807700,29.633300,7.200820,...,4.987470,7.932460,8.338280,0.000000,24.001700,8.907420,7.349950,42.84740,transcript_190157,ENSG00000277400.1
